# Explore RNAseq of FAC Sorted retinal cells from 42hpf zebrafish ([Nerli, Rocha-Martins, Norden, 2022](https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE194158))

This notebook can be accessed here:  
[angueyranih.github.io/drRNAseq/lab](angueyranih.github.io/drRNAseq/lab)  

## Norden Lab dataset available at:  
https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE194158

From the authors
> "This dataset was acquired to identify the gene expression profile of developing retinal cell types at 42 hours post fertilization (hpf): retinal ganglion cells, photoreceptors, inhibitory neurons (amacrine cells and horizontal cells) and progenitor cells.  
> Retinal cell types were sorted from dissociated zebrafish retinas at 42 hpf using the triple transgenic line Tg(ath5:gap-RFP), Tg(crx:gap-CFP), Tg(ptf1a:gal4,UAS:gap-YFP). For each of the 5 replicates, 25 retinas were pooled"

## Instructions to explore dataset:  
https://github.com/angueyraNIH/drRNAseq

----
----

### How to use the notebook
- To explore the datasets, you will need to run the code in the cells below.  
- To run a cell, you need to click unto it to select it and press "Ctrl"+"Enter" or the small *play* symbol in the navigation bar on top of this notebook.  
- Cells that are currently running display [\*] on the left.  
- Cells that have finished running will display a number instead (e.g.[8])
---
---
#### Saving plots:
- Right click on the image and click on "Create New View for Output"
- Right click on image in Output View panel and click on "Save As..."

#### First: load python environment

In [ ]:
import pyolite
print("Successfully loaded pyolite version {0}".format(pyolite.__version__))

#### Second: load all necessary extensions (this may take a few minutes...)

In [ ]:
print("Loading extensions...")
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.font_manager as font_manager
from scipy.stats import zscore
import os
import pyodide
import piplite

await piplite.install('ipywidgets')
import ipywidgets
pBGMenu = ipywidgets.Dropdown(options=['Light', 'Dark'],value='Dark',disabled=False, layout=ipywidgets.Layout(width='20%', height='40px'))
pBGLabel = ipywidgets.widgets.Label('Select plotting style', layout=ipywidgets.Layout(width='20%', height='30px'))
pBG = ipywidgets.widgets.VBox([pBGLabel,pBGMenu])
heatmapNorm = ipywidgets.Checkbox(value=False, description='Heatmap normalization', disabled=False)


# Load plotting function
URL = 'https://raw.githubusercontent.com/angueyraNIH/jupyterLiteDemo/main/content/juanPlot-0a2-py2.py3-none-any.whl'
await piplite.install(URL)
from juanPlot import *
print("Successfully loaded extensions!")

# load datasets
print("Loading datasets...")
URL = 'https://raw.githubusercontent.com/angueyraNIH/drRNAseq/main/content/data/Nerli2022_42hpfRet.csv'
zfN = pd.read_csv(pyodide.open_url(URL))
print("\t Nerli et al. (2022)")


print("Successfully loaded dataset(s)!\n\n")

# Display selector for plotting background:
pBG

#### Define some plotting bases and colors

In [ ]:
# photoreceptor Colors
pC_42hpf = {
    'RPC' : '#DADADA', # Retinal progenitor cell
    'PR' : '#dcc360', # Photoreceptor cell (developing)
    'HC_AC' : '#3DF591', # Horizontal and Amacrine cell (developing)
    'RGC' : '#F53D59', # Retinal Ganglion cell (developing)
}

# default parameters for plotting
if pBGMenu.value=='Dark':
    # dark background
    plt.style.use('dark_background')
    params = {"ytick.color" : "w", "xtick.color" : "w",
              "axes.labelcolor" : "w", "axes.edgecolor" : "w",
              "axes.linewidth" : 3,
              "xtick.major.width" : 3, "ytick.major.width" : 3,
              "xtick.minor.width" : 3, "ytick.minor.width" : 3,
              "xtick.major.size" : 8, "ytick.major.size" : 8,
              "xtick.minor.size" : 8, "ytick.minor.size" : 8,
              "text.color" : "w"}
    plt.rcParams.update(params)
    baseColor = '#ffffff' #white
else:
    # white background
    plt.style.use('default')
    params = {"ytick.color" : "k", "xtick.color" : "k",
              "axes.labelcolor" : "k", "axes.edgecolor" : "k",
              "axes.linewidth" : 3,
              "xtick.major.width" : 3, "ytick.major.width" : 3,
              "xtick.minor.width" : 3, "ytick.minor.width" : 3,
              "xtick.major.size" : 8, "ytick.major.size" : 8,
              "xtick.minor.size" : 8, "ytick.minor.size" : 8,
              "text.color" : "k"}
    plt.rcParams.update(params)
    baseColor = '#000000' #black
    


print("Plotting style is: {0}".format(pBGMenu.value))

***
# Index <a id='Index'></a>
- [Bar plot for single gene: Nerli et al. (2022)](#barPlot42hpf)
- [Heatmap for gene family: Nerli et al. (2022)](#heatMap42hpf)

***
## Bar plot for single gene: provide gene symbol (e.g. '_tbx2a_')<a id='barPlot42hpf'></a>
***
[Back to Index](#Index)

In [ ]:
geneSymbol = 'tbx2b' # replace geneSymbol here and rerun cell to replot

barData = zfN[zfN['symbol']==geneSymbol] #get data (should add a check here with isin)
fH, axH = plt.subplots(figsize=(8,6))
pH = plotBars_Nerli2022(barData, geneSymbol, ax=axH, pC=pC_42hpf)
plt.subplots_adjust(left=0.15, right=.95, top=0.90, bottom=0.1)
plt.tight_layout()
plt.show()

***
## HeatMap for gene family: provide gene symbol prefix (e.g. '_opn1_')<a id='heatMap'></a>
> retrieves all genes whose symbol starts with defined geneSymbol  
***

[Back to Index](#Index)

In [ ]:
# display option for heatmap normalization
heatmapNorm

In [ ]:
geneSymbol='meis' # replace geneSymbol here and rerun cell to replot

heatmapDataN = zfN[zfN['symbol'].str.startswith(geneSymbol)]
heatmapDataN = heatmapDataN.sort_values(by=["symbol"])
fH, axH = plt.subplots(figsize=(20,(0.5*heatmapDataN.shape[0])+5))
hmH, cbH = heatmap_Nerli2022(heatmapDataN, pC = pC_42hpf, norm=heatmapNorm.value)
plt.tight_layout()
plt.show()

***
## HeatMap for gene list: provide gene symbols separated by "|" (e.g. '_tbx2a_|_tbx2b_|_foxq2_')<a id='heatMapList'></a>
> retrieves all genes in list and creates heatmap  
***

[Back to Index](#Index)

In [ ]:
# display option for heatmap normalization
heatmapNorm

In [ ]:
geneSymbol='tbx2a|tbx2b|foxq2' # replace geneSymbol here and rerun cell to replot

heatmapDataN = zfN[zfN['symbol'].str.contains(geneSymbol)]
heatmapDataN = heatmapDataN.sort_values(by=["symbol"])
fH, axH = plt.subplots(figsize=(20,(0.5*heatmapDataN.shape[0])+5))
hmH, cbH = heatmap_Nerli2022(heatmapDataN, pC = pC_42hpf, norm=heatmapNorm.value)
plt.tight_layout()
plt.show()